In [7]:
import tkinter as tk
from tkinter import ttk
from tkinter import messagebox
import re
import sys
my_weight = [0.3,0.2,0.4,0.1]



class Error(Exception):
    pass
class IncompleteFormError(Error):
    pass

class Initialize():
    """Class to initilize the original prompt window for input of Candidate's information. An error message box will pop up if 
    missing answer"""
    def __init__(self):
        #initialize fields needed
        self.basic_info = {"Name":None,"Age":None,"Gender":None,"Marital Status":None,"Highest Education":None,"Employment Status":None}
        self.app_info = {"Height in Inches":None,"Weight in Pounds":None}
        self.emp_info = {"Industry of Profession":None,"Years of Work Experience":None}
        self.int_info = {"Hobbies":None}
        self.fam_info1 = {"Parent1's Highest Education":None}
        self.fam_info2 = {"Parent2's Highest Education":None}
        self.field_list = list(self.basic_info.keys())+list(self.app_info.keys())+list(self.emp_info.keys())+list(self.int_info.keys())\
                +list(self.fam_info1.keys())+list(self.fam_info2.keys())
        #build prompt window
        self.window = tk.Tk()
        self.window.title("Welcome")
        self.ents = self.makeform(self.window,self.field_list)  
        submit = tk.Button(self.window, text='submit', command=(lambda e=self.ents: self.fetch_entry(e)))
        submit.pack(side=tk.LEFT, padx=5, pady=5)
        ext = tk.Button(self.window, text='Close', command=self.window.destroy)
        ext.pack(side=tk.LEFT, padx=5, pady=5)
        self.window.mainloop()
        
    def makeform(self, window, fields):
        """method to list out the fields on the prompt window, by rows, with entries stored in entries list as(field, entry)"""
        #for fields where options are available, a dropdown entry will be shown
        dropdown = {"Gender":["Male","Female"],"Marital Status":["Single","Married","Divorced","In a Relationship"],
            "Highest Education":["Bachelor's Degree","Master's Degree", "Doctoral Degree","High School Diploma or below"],
            "Employment Status":["Employed","Unemployed"],
            "Industry of Profession":["Accounting & Legal","Business Services","Finance","Health Care","Information Technology"
                                    ,"Manufacturing","Media","Restaurants, Bars & Food Services","Retail"],
            "Parent1's Highest Education":["Bachelor's Degree","Master's Degree", "Doctoral Degree","High School Diploma or below"],
            "Parent2's Highest Education":["Bachelor's Degree","Master's Degree", "Doctoral Degree","High School Diploma or below"]}
        entries = []
        for field in fields:
            row = tk.Frame(window)
            lab = tk.Label(row, width=20, text=field, anchor="w")
            if field in dropdown.keys():
                ent = ttk.Combobox(row, values=dropdown[field])
            else:
                ent = tk.Entry(row)
            row.pack(side=tk.TOP, fill=tk.X, padx=5, pady=5)
            lab.pack(side=tk.LEFT)
            ent.pack(side=tk.RIGHT, expand=tk.YES, fill=tk.X)
            entries.append((field, ent))
        return entries
    
    
    def fetch_entry(self,entries):
        """method to store entry answers as dictionary values in their respective dictionaries"""
        try:
            for entry in entries:
                field = entry[0]
                answer = entry[1].get()
                if answer == "":
                    raise IncompleteFormError
                    break
                if field in self.basic_info.keys():
                    self.basic_info[field] = answer
                if field in self.app_info.keys():
                    self.app_info[field] = answer
                if field in self.emp_info.keys():
                    self.emp_info[field] = answer
                if field in self.int_info.keys():
                    hobby_input = re.split('[^a-zA-Z]', answer)
                    #sometimes two consequetive deliminator will create an empty string element. if so, remove
                    if "" in hobby_input:
                        hobby_input.remove("")
                    self.int_info[field] = [ele.capitalize() for ele in hobby_input]               
                if field in self.fam_info1.keys():
                    self.fam_info1[field] = answer
                if field in self.fam_info2.keys():
                    self.fam_info2[field] = answer
        #If fields are left unanswered, an error message will show
        except IncompleteFormError:
            messagebox.showinfo("Error", "Complete all fields")

class Person():
    """basic information about the candidate, also has a method to check for validity"""
    def __init__(self, basic_info):
        #super().__init__(basic_info)
        self.name = basic_info["Name"]
        self.age = int(basic_info["Age"])
        self.gender = basic_info["Gender"]
        self.marital = basic_info["Marital Status"]
        self.edu = basic_info["Highest Education"]
        self.emp = basic_info["Employment Status"]
    
    #method to check validity
    def check(self): 
        boo = True
        if self.gender != "Male":
            boo = False
        if self.marital != "Single":
            boo = False
        if self.edu not in ["Bachelor's Degree","Master's Degree", "Doctoral Degree"]:
            boo = False
        if self.age not in range(30,41):
            boo = False
        if self.emp != "Employed":
            boo = False
        return boo
    
class App():
    """Appearance that has attributes about the candidate's attractivenss, as well as a function to calculate the candidate's BMI. 
    Both are driving factors for the score method"""
    def __init__(self,app_info):
        self.height = float(app_info["Height in Inches"])
        self.weight = float(app_info["Weight in Pounds"])
        self.__attractiveness = 50        

    def cal_BMI(self):
        """BMI is calculated as weight in pounds/height in inches square multipled by 703"""
        return self.weight/((self.height)**2)*703
        
        #Since attractivenss is a variable that will be changed according to feedback. Using getter and setting to control access
    @property
    def attractivenss(self):
        return self.__attractiveness
        
    @attractivenss.setter
    def attractiveness(self, arg):
        """Attractiveness attribute can be changed based on feedback, whether Plus, Minus or nothing"""
        if arg == "plus":
            self.__attractiveness += 10
        if arg == "minus":
            self.__attractiveness -= 10
        else:
            pass
        
    def BMI_score(self):
        BMI_score = 0
        if self.cal_BMI() >= 18.5 and self.cal_BMI() <= 20.5:
            BMI_score = 50
        if self.cal_BMI() > 20.5 and self.cal_BMI() <= 23.0:
            BMI_score = 30
        if self.cal_BMI() > 23.0 and self.cal_BMI() <= 25.0:
            BMI_score = 10
        else:
            BMI_score = 0
        return BMI_score
            
    def score(self):
        return self.BMI_score() + self.__attractiveness

class Employment():
    """Class about the candidate's employment. Attributes are Candiate's professional industry and years of experience. 
    Both attributes are driving factors for the score method"""
    def __init__(self,emp_info):
        self.ind = emp_info["Industry of Profession"]
        self.exp = int(emp_info["Years of Work Experience"])

    def ind_score(self):
        ind_score = 0
        if self.ind in ["Accounting & Legal","Business Services","Finance","Information Technology" ]:
            ind_score += 60
        else:
            ind_score += 30
        return ind_score
  
    def exp_score(self):
        exp_score = 0          
        if self.exp <=5:
            exp_score += 20
        elif self.exp <=10:
            exp_score += 30
        else:
            exp_score += 40
        return exp_score
        
    def score(self):
        return self.ind_score() + self.exp_score()
        
class Interests():
    """Interest class that collects candidate's interest, scores based on common interest. method includes adding interests through setter"""
    def __init__(self, int_info):
        self.__interest = int_info["Hobbies"] #list of initial interests
        self.__pref_int = ["Skiing","Reading","Hiking"]
        
    #since common interest will be added outside the class, using getter and setter for access control
    @property
    def interest(self):
        return self.__interest
    
    @interest.setter
    def interest(self,new_activity):
        new_interest = new_activity.capitalize()
        if new_interest not in self.__interest and new_interest not in self.__pref_int:#the two discovered a new common hobby
            self.__interest.append(new_interest)
            self.__pref_int.append(new_interest)
        if new_interest in self.__interest and new_interest not in self.__pref_int:#candidate introduced Kayla to a new hobby
            self.__pref_int.append(new_interest)
        if new_interest not in self.__interest and new_interest in self.__pref_int:#Kayla introduced candidate to a new hobby
            self.__interest.append(new_interest)
        
    def score(self):
        int_score = 0
        #candidate receives base score if he has more than 3 hobbies
        if len(self.__interest) >= 3:
            int_score += 40
        #every common hobby receiveds extra score. when new hobbies are added during program, they will also receive credit
        common_int = len(list(set(self.__interest)&set(self.__pref_int)))
        if common_int >=1:
            int_score += 20*common_int
        return int_score

class FamilyMember():
    """family member class that will asign a score if candidate's parents are college educated or above"""
    def __init__(self, family_info):
        self.edu = list(family_info.values())[0]
    
    def score(self):
        fam_score = 0
        if self.edu in ["Bachelor's Degree","Master's Degree", "Doctoral Degree"]:
            fam_score += 60
        else:
            fam_score += 20
        return fam_score

class Candidate():
    """Class that contains candidate's attributes, and scores"""
    def __init__(self, basic_info,app_info,emp_info,int_info,fam_info1, fam_info2):
        self.basic = Person(basic_info)
        self.look = App(app_info)
        self.job = Employment(emp_info)
        self.hobby = Interests(int_info)
        self.parent1 = FamilyMember(fam_info1)
        self.parent2 = FamilyMember(fam_info2)
 
    def initial_score(self,weight):
        """intial score method that returns a score based on initial criteria. >75 will kickoff meet up"""
        raw_score = [self.look.score(),self.job.score(),self.hobby.score(),self.parent1.score()+self.parent2.score()]
        ini_score = sum(map(lambda x,y: x*y,weight,raw_score))
        #print(raw_score)
        return ini_score
    
    def first_score(self,weight,cmt,adjustment=""):
        """during first meet up, the user can add/subtract bonus, add comment, and adjust appearance"""
        self.look.attractiveness = adjustment
        raw_score = [self.look.score(),self.job.score(),self.hobby.score(),self.parent1.score()+self.parent2.score()]
        score1 = sum(map(lambda x,y: x*y,weight,raw_score))
        #print(raw_score)
        return score1
    def date_score(self, weight,cmt,new_interest=None):
        """during second to fifth meet up, the user can add/subtract bonus, add comment, and add a common interest if tried
        the activity with candidate"""
        if new_interest != None:
            self.hobby.interest = new_interest
        raw_score = [self.look.score(),self.job.score(),self.hobby.score(),self.parent1.score()+self.parent2.score()]
        score2 = sum(map(lambda x,y: x*y,weight,raw_score))
        #print(raw_score)
        return score2  

def matchmaking_func():
    """main function to trigger the interactive program
    1.prompt for intial input
    2.return initial values
    3.input prompt for feedbacks
    4.relative actions: wether select another candidate, continue dating, or find a perfect match"""
    person= Initialize()
    #John is the object for a candidate
    John = Candidate(person.basic_info,person.app_info,person.emp_info,person.int_info,person.fam_info1,person.fam_info2)
    score_track = {} #dictionary to store scores and comments, for track record
    pointer = True #boolean to kickoff restart if needed
    screen_score =  John.initial_score(my_weight) #score from initial screenning
    #dating only start when 1. hard criteria(such as marital status) is met 2.initial screen >=75
    if not John.basic.check():
        #print("Does not meet criteria")
        messagebox.showinfo("Error", "Does not meet criteria")
        pointer = False
    elif screen_score < 75:
        print("initial screening returns ",screen_score,"start over")
        pointer = False
    else: #if initial score passes threshold, can start date
        print("initial screening returns ",screen_score,"He seems to be a good fit, you should meet up")
        score_track["pre_screen"] = [screen_score]
        done = False #boolean to help end program if a perfect match is found (>100 score)
        while pointer:
            pre_bonus = 0 #bonus points are accumulative
            for date_count in range(1,7):
            #first date asks for adjustment on appearance
                if date_count == 1:
                    #feedback for 1st date: feedback and adjustment on appearance now they have met
                    arg1 = input("what's your feedback? (optional) ")
                    arg2 = input("how does he look? (plus, minus or skip) ")
                    fst_score = John.first_score(my_weight,arg1,arg2)
                    #whenever score falls below 75, program restarts
                    if fst_score < 75:
                        print("Score",fst_score," falls below 75, start over")
                        pointer = False
                        break #condition to break inner loop
                    else:
                        print("Score ",fst_score,". It seems to be working well...")
                        score_track["date No.1"] = [fst_score,arg2]
                #if all five dates are completed, score hasn't fallen before 75 or above 100
                elif date_count == 6:
                    avg_score = sum([x[0] for x in list(score_track.values())])/6 
                    if avg_score >= 90.0:
                        print("Average Score ",avg_score,". You are a perfect match with", John.basic.name)                        
                    else:
                        print("Let's take a break and try again")
                    #innerloop at the end, natually breaks
                    done = True #condition to break while loop
               #for dates 2-5, only variable is bonus points and new interest additions
                else:
                    bonus = pre_bonus + float(input("points to add (- for subtract)(required) ")) #need to make sure prior bonus still counts
                    arg2 = input("what's your feedback? (optional) ")
                    arg3 = input("what activity did you enjoy with him (optional) ")
                    #if a new activity is tried, it will be added a new interest score calculated along with bonus
                    if len(arg3) > 0:
                        dt_score = John.date_score(my_weight,arg2,arg3) + bonus
                    else:
                        dt_score = John.date_score(my_weight,arg2) + bonus
                    if dt_score < 75:
                        print("Score ",dt_score," falls below 75, start over")
                        pointer = False
                        break #condition to break inner loop
                    elif dt_score > 100:
                        print("Score ",dt_score,". You're a perfect match with", John.basic.name)
                        done = True #condition to break while loop
                        break #condition to break inner loop
                    else:
                        print("Score ",dt_score,". It seems to be working well")
                        score_track["date No."+str(date_count)] = [dt_score,arg2]
                        pre_bonus = bonus
            if done:
                break
    return pointer #pointer will be True if program runs till the 5th date, and False if it falls below 75 before reaching 5th date

#Main code of the program
result = matchmaking_func()
#keep selecting individuals until finding a match
while result == False:
    result = matchmaking_func()
    

TypeError: int() argument must be a string, a bytes-like object or a number, not 'NoneType'